In [14]:
import pandas as pd
import html2text
import module.veille_sociale as vs
import importlib
from tqdm import tqdm
from docx import Document
from docx.enum.text import WD_COLOR_INDEX
import os
import html2text

In [9]:
importlib.reload(vs)

<module 'module.veille_sociale' from '/home/mathildedacruz/amendements/module/veille_sociale.py'>

In [10]:
nlp = vs.SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
nlp_lg = vs.fr_core_news_lg.load()

In [2]:
amd = pd.read_csv('amendements.csv', sep=';')
log = pd.read_csv('logs.csv', sep=';')

/tmp/ipykernel_479648/475057402.py:1: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  amd = pd.read_csv('amendements.csv', sep=';')
/tmp/ipykernel_479648/475057402.py:2: DtypeWarning: Columns (20,21,22,26,33) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv('logs.csv', sep=';')


In [3]:
text = html2text.html2text(log[log['id_amdt']==12907].iloc[0]['corps'])

In [4]:
print(text)

I. - L'article 238 _bis_ du code general des impots est complete par un 7
ainsi redige :

« 7\. Ouvrent droit a une reduction d'impot egale a 80 % de leur montant les
versements effectues sans limite par les petites et moyennes entreprises dont
le chiffre d'affaires est inferieur a 50 Millions d'euros au profit des
organismes ci-dessus mentionnes a condition que ces versements soient affectes
a la rehabilitation, restauration, conservation, entretien de monuments
historiques. »

II. - La perte de recettes pour l'État est compensee a due concurrence par la
creation d'une taxe additionnelle aux droits mentionnes aux articles 575 et
575 A du code general des impots.




In [5]:
import torch

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM
)

barthez_tokenizer = AutoTokenizer.from_pretrained("moussaKam/barthez")
barthez_model = AutoModelForSeq2SeqLM.from_pretrained("moussaKam/barthez-orangesum-abstract")

In [11]:
exposes = list(amd['expose'])

In [12]:
text = html2text.html2text(amd[amd['id']==15463]['expose'].iloc[0])

In [13]:
def summarize_docx(text, titre = None, comm=None, dictionary = None, n_sentences=5, ratio=0, nlp=None, method='local'):
   
    sentences, summary, sentences_id = vs.summarize_tr(text, dictionary, n_sentences, ratio, method, nlp=nlp)

    doc_result = Document()
    paragraph = doc_result.add_paragraph()
    for i in range(len(sentences)):
        sentence = sentences[i]
        run = paragraph.add_run(sentence.strip())
        if i in sentences_id:
            run.bold = True
            run.font.highlight_color = WD_COLOR_INDEX.YELLOW #surligner les phrases du résumé   
    
    doc_result.save('/home/mathildedacruz/amendements/test.docx')
    
    return summary

In [15]:
summarize_docx(html2text.html2text(exposes[1243]), dictionary = None, n_sentences=1, method='local', nlp=nlp_lg)  

['Au regard des besoins sociaux et des programmes d’action des pouvoirs publics, cet amendement propose donc d’apporter une clarification et une simplification du statut fiscal des opérations de construction et de rénovation dans ces domaines mis en œuvre notamment par le secteur associatif et caritatif.']

In [16]:
summarize_docx(text, dictionary = None, n_sentences=1, method='local', nlp=nlp_lg)  

['\\- accompagner l’agriculture dans ses transitions économiques sociétales et environnementales  \\- créer plus de valeur dans les territoires via des projets territoriaux associant les agriculteurs, les collectivités et la société civile (gestion de l’eau, du foncier, la conversion en bio ...)  \\- restaurer le dialogue entre la société et l’agriculture  Des actions phares ont été définies (assistance pour la transmission des exploitations, accompagner les agriculteurs vers le bio ...) mais si la baisse du financement des Chambres d’agriculture se confirmait, ces actions indispensables pour les agriculteurs, pour le développement et le dynamisme des territoires ruraux ne pourraient être mises en place et se trouveraient gravement remises en cause à l’heure où elles sont pourtant les plus nécessaires.']